In [1]:
import numpy as np
import quandl
import pandas as pd
import datetime
from pandas.tseries.offsets import BDay
import MySQLdb
from stock_loader import StockLoader


from alpha_vantage.techindicators import TechIndicators
from alpha_vantage.timeseries import TimeSeries

In [3]:
quandl.ApiConfig.api_key = '4objFXpTN7u4cnsvX6Fa'
data = quandl.get("WIKI/BODY")

In [40]:
data.loc['2014-07-10']

Open                   NaN
High                0.6150
Low                 0.5250
Close               0.6079
Volume         586115.0000
Ex-Dividend         0.0000
Split Ratio         1.0000
Adj. Open              NaN
Adj. High           0.6150
Adj. Low            0.5250
Adj. Close          0.6079
Adj. Volume    586115.0000
Name: 2014-07-10 00:00:00, dtype: float64

In [21]:
ts = TimeSeries(key='92FRTVHCC356S9ZS',output_format='pandas')
ts_raw, ts_meta_data = ts.get_daily_adjusted(symbol='AAPL',outputsize='full')

In [24]:
ts_raw.loc["2017-01-03"]

6. volume               2.878186e+07
3. low                  1.147600e+02
1. open                 1.158000e+02
7. dividend amount      0.000000e+00
8. split coefficient    1.000000e+00
2. high                 1.163300e+02
5. adjusted close       1.138447e+02
4. close                1.161500e+02
Name: 2017-01-03, dtype: float64

In [5]:
ts_meta_data

{'1. Information': 'Daily Prices (open, high, low, close) and Volumes',
 '2. Symbol': 'SPY',
 '3. Last Refreshed': '2018-02-09',
 '4. Output Size': 'Full size',
 '5. Time Zone': 'US/Eastern'}

In [1]:
db=MySQLdb.connect(host='localhost', user='root', passwd='eboue',
                   db='finance', port=3306)

In [19]:
sql_format = 'SELECT * FROM StockPriceVolume WHERE Symbol=\'%s\' AND Date BETWEEN \'%s\' AND \'%s\' ORDER BY Date'
a = datetime.date.today() - datetime.timedelta(days=20)
b = a - datetime.timedelta(days=10)
sql = sql_format % ('AAPL', b, a)
pd.read_sql(sql, con=db)

"SELECT * FROM StockPriceVolume WHERE Symbol='AAPL' AND Date BETWEEN '2018-02-16' AND '2018-02-26' ORDER BY Date"

In [2]:
loader = StockLoader()

In [9]:
result = loader.get_price_volume('AAPL', datetime.date(2000, 1, 1),
                               datetime.date(2017, 2, 1))
result['DayChange'] = result['Close']/result['Open'] - 1
result['DayAmplitude'] = (result['High'] - result['Low'])/result['Open']
result['DayChangeSign'] = result['Close'] > result['Open']

In [7]:
result

0        NaN
1     116.15
2     116.02
3     116.61
4     117.91
5     118.99
6     119.11
7     119.75
8     119.25
9     119.04
10    120.00
11    119.99
12    119.78
13    120.00
14    120.08
15    119.97
16    121.88
17    121.94
18    121.95
19    121.63
20    121.35
Name: Close, dtype: float64

In [8]:
result

,Symbol,Date,Open,High,Low,Close,AdjOpen,AdjHigh,AdjLow,AdjClose,Volume,HasEvent,DayChange,DayAmplitude,DayChangeSign
0,AAPL,2017-01-03,115.800,116.3300,114.7600,116.15,114.369701,114.893155,113.342546,114.715378,28781865.0,0,0.003022,0.013558,True
1,AAPL,2017-01-04,115.850,116.5100,115.7500,116.02,114.419083,115.070931,114.320318,114.586983,21118116.0,0,0.001467,0.006560,True
2,AAPL,2017-01-05,115.920,116.8642,115.8100,116.61,114.488219,115.420756,114.379577,115.169696,22193587.0,0,0.005952,0.009094,True
3,AAPL,2017-01-06,116.780,118.1600,116.4700,117.91,115.337596,116.700551,115.031425,116.453639,31751900.0,0,0.009676,0.014472,True
4,AAPL,2017-01-09,117.950,119.4300,117.9400,118.99,116.493145,117.954865,116.483269,117.520300,33561948.0,0,0.008817,0.012632,True
5,AAPL,2017-01-10,118.770,119.3800,118.3000,119.11,117.303017,117.905483,116.838822,117.638817,24462051.0,0,0.002863,0.009093,True
6,AAPL,2017-01-11,118.740,119.9300,118.6000,119.75,117.273388,118.448689,117.135117,118.270913,27588593.0,0,0.008506,0.011201,True
7,AAPL,2017-01-12,118.895,119.3000,118.2100,119.25,117.426473,117.826471,116.749934,117.777088,27086220.0,0,0.002986,0.009168,True
8,AAPL,2017-01-13,119.110,119.6200,118.8100,119.04,117.638817,118.142518,117.342523,117.569682,26111948.0,0,-0.000588,0.006800,False
9,AAPL,2017-01-17,118.340,120.2400,118.2200,120.00,116.878328,118.754860,116.759810,118.517825,34439843.0,0,0.014027,0.017069,True
